# DE-Sim tutorial

We show how to use DE-Sim to build and simulate discrete-event models.

First, use `pip` to install `de_sim`.

In [1]:
!pip install de_sim

     |████████████████████████████████| 48 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 175 kB 8.2 MB/s eta 0:00:01
  Created wheel for pympler: filename=Pympler-0.8-py3-none-any.whl size=164713 sha256=72ff16f34d5b9b4c28a5867327d94193868bca6a965d79eeacfb1d5d9a45a501
  Stored in directory: /root/.cache/pip/wheels/a0/a9/99/337816ce8e8acc5b1849abe49c8f637a9be9a5005f72318ecf
Successfully built pympler
ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__

In [2]:
from de_sim.simulation_engine import SimulationEngine
from de_sim.simulation_message import SimulationMessage
from de_sim.simulation_object import ApplicationSimulationObject

An OO DES application that uses DE-Sim can be defined in three steps:

**1: Create event message types by subclassing `SimulationMessage`, a customized DE-Sim class.**

An event message is sent when an event is scheduled. For some events, the type of the event message provides all the information required to execute the event, as illustrated by event message class `MessageSentToSelf`. For other events, the event message contains data in one or more attributes, as illustrated by `MessageWithAttribute`.

Also, an event message class must be documented by a docstring.

In [3]:
class MessageSentToSelf(SimulationMessage):
    "A message type with no attributes"

class MessageWithAttribute(SimulationMessage):
    'An event message type with an attribute'
    attributes = ['attr_1']

**2: Define simulation application object types by subclassing `ApplicationSimulationObject`.**

Multiple instances of a simulation object type can be instantiated and used in a DE-Sim simulation.
A simulation object is like a thread, because each object executes independently.
But discrete-event simulation (DES) objects are scheduled differently than threads.
Whereas threads run whenever they have work to do,
DES schedules objects according to a scheduling rule that honors temporal causality:

1. Events are executed in non-decreasing time order. (The tie-breaking rules for execution of events at equal simulation time are discussed below.) 

The causality rule has two immediate consequences:

1. Each simulation object executes its events in increasing time order.
2. All synchronization between simulation objects is controlled by the simulation times of events.

The simulation object type `SimpleSimulationObject` below illustrates the key features of DE-Sim's `ApplicationSimulationObject`.

In [4]:
class SimpleSimulationObject(ApplicationSimulationObject):
    """ A simple example DE-Sim simulation object type

    Attributes:
        name: The name of an instance of this object; each instance must have a unique name.
        delay: A float, which provides the delay between events.
    """

    def __init__(self, name, delay):
        """ Initialize a simulation object

        Args:
            name: The unique name of this instance of this object.
            delay: The delay between events.
        """
        super().__init__(name)
        self.delay = delay

    def send_initial_events(self):
        """ Send the initial events for this object; called by the simulator
        """
        self.send_event(self.delay, self, MessageSentToSelf())

    def handle_simulation_event(self, event):
        """ Handle a simulation event

        Args:
            event: The DE-Sim event being executed. Not used in this example.
        """
        self.send_event(self.delay, self, MessageSentToSelf())

    # event_handlers is a list of pairs that maps each event message type
    # received by this simulation object type to the method that handles
    # the event message type
    event_handlers = [(MessageSentToSelf, handle_simulation_event)]

    # messages_sent registers all message types sent by this object
    messages_sent = [MessageSentToSelf]

A subclass of `ApplicationSimulationObject` contains special methods and attributes that define its simulation behavior.

* Methods
  1. `send_initial_events` (optional): a method named `send_initial_events` sends a simulation object's initial events. After all objects are loaded into a simulation, the simulator calls each object's `send_initial_events` method before starting a simulation. A simulation must send at least one initial event to initiate a model's dynamics.
  2. event handlers: an event handler is a method that handles a simulation event. Event handlers have the signature `event_handler(event)`, where `event` is a DE-Sim simulation event (`de_sim.event.Event`). A subclass of `ApplicationSimulationObject` must define at least one event handler. `handle_simulation_event` is an event handler in the example above.
* Attributes
  1. `event_handlers`: the attribute `event_handlers` must contain a list of pairs that maps each event message type received by a subclass of `ApplicationSimulationObject` to the subclass' method that handles the event message type. In the example above, `event_handlers` associates `MessageSentToSelf` event messages with the `handle_simulation_event` event handler. When a simulation object executes an event, the simulator vectors the event's message to the handler associated with its type.
  2. `messages_sent`: the types of messages sent by a subclass of `ApplicationSimulationObject` are listed in `messages_sent`. It is used to ensure that a simulation object doesn't send messages of the wrong type.

To schedule events, `ApplicationSimulationObject` provides the method

    send_event(delay, receiving_object, event_message)
    
which schedules an event to occur `delay` time units in the future at simulation object `receiving_object`, which will execute a simulation event containing `event_message`. An event can be scheduled for any simulation object in a simulation, including the object scheduling the event, as shown in the example above.
Object-oriented DES terminology also describes the event message as being sent by the sending object at the message's send time (the simulation time when the event was scheduled) and received by the receiving object at the event's receive time (the simulation time when the event is executed).

`event_message` must be an instance of a `SimulationMessage`, and may have attributes that contain data used by the event.
The event will be executed by an event handler in simulation object `receiving_object`,
with a parameter that is set to a simulation event containing `event_message` at its scheduled simulation time.
The example above illustrates this in its

    handle_simulation_event(self, event)

method.
In this example all simulation events are scheduled to be executed by the object that creates the event, but realistic
simulations contain multiple simulation objects which schedule events for each other.

**3: Execute a simulation by creating a `SimulationEngine`, instantiating the application objects, sending their initial event messages, and running the simulation.**

In [7]:
# create a simulation engine
simulation_engine = SimulationEngine()

# create a simulation object and add it to the simulation
simulation_engine.add_object(SimpleSimulationObject('object_1', 6))

# initialize and run the simulation for 100 time units
simulation_engine.initialize()
num_events = simulation_engine.run(100).num_events
num_events

16

# DE-Sim benchmark example

The parallel hold (PHOLD) model is used to benchmark parallel DES (PDES) simulators.
We implement PHOLD as a DE-Sim model and use it to illustrate these features:

* Run multiple instances of a simulation object type
* Use multiple `SimulationMessage` types
* Run a stochastic simulation
* Record a simulation's predictions
* Use the `de_sim.event.Event` object passed to every event handler
* Use `self.time` to access the current simulation time

In [30]:
""" Parallel hold (PHOLD) model commonly used to benchmark parallel discrete-event simulators :cite:`fujimoto1990performance`.

:Author: Arthur Goldberg <Arthur.Goldberg@mssm.edu>
:Date: 2016-06-10
:Copyright: 2016-2020, Karr Lab
:License: MIT
"""

import random


class MessageSentToSelf(SimulationMessage):
    "A message that's sent to self"


class MessageSentToOtherObject(SimulationMessage):
    "A message that's sent to another PHold simulation object"


class InitMsg(SimulationMessage):
    'initialization message'


MESSAGE_TYPES = [MessageSentToSelf, MessageSentToOtherObject, InitMsg]


class PholdSimulationObject(ApplicationSimulationObject):
    """ Run a PHOLD simulation

    Attributes:
        args: a :obj:`Namespace` that defines:
            `num_phold_objects`: the number of PHOLD objects to run
            `frac_self_events`: the fraction of events sent to `self`
            `time_max`: the end time for the simulation
    """
    def __init__(self, name, args):
        self.args = args
        super().__init__(name)

    def send_initial_events(self):
        self.send_event(random.expovariate(1.0), self, InitMsg())

    @staticmethod
    def record_event_header():
        print('\t'.join(('Sender',
                         'Send',
                         "Receivr",
                         'Event',
                         'Message type')))
        print('\t'.join(('', 'time', '', 'time', '')))
        
    def record_event(self, event):
        record_format = '{}\t{:.2f}\t{}\t{:.2f}\t{}'
        print(record_format.format(event.sending_object.name,
                                   event.creation_time,
                                   event.receiving_object.name,
                                   self.time,
                                   type(event.message).__name__))

    def handle_simulation_event(self, event):
        """ Handle a simulation event """
        # Record this event
        self.record_event(event)
        # Schedule an event
        if random.random() < self.args.frac_self_events or \
            self.args.num_phold_objects == 1:
            receiver = self
        else:
            # Send the event to another randomly selected object
            # Pick an object index in [0, num_phold-2], and increment if self or greater
            obj_index = random.randrange(self.args.num_phold_objects - 1)
            if int(self.name) <= obj_index:
                obj_index += 1
            receiver = self.simulator.simulation_objects[str(obj_index)]

        if receiver == self:
            message_type = MessageSentToSelf
        else:
            message_type = MessageSentToOtherObject
        self.send_event(random.expovariate(1.0), receiver, message_type())

    event_handlers = [(sim_msg_type, 'handle_simulation_event') \
                      for sim_msg_type in MESSAGE_TYPES]

    # register the message types sent
    messages_sent = MESSAGE_TYPES


def create_and_run(args):

    # create a simulator
    simulator = SimulationEngine()

    # create simulation objects, and send each one an initial event message to self
    for obj_id in range(args.num_phold_objects):
        phold_obj = PholdSimulationObject(str(obj_id), args)
        simulator.add_object(phold_obj)

    # run the simulation
    simulator.initialize()
    PholdSimulationObject.record_event_header()
    event_num = simulator.simulate(args.time_max).num_events
    print("Executed {} events.\n".format(event_num))

The PHOLD model runs n identical `PholdSimulationObject` objects.
To simplify the example, each object's name is the string representation of its index.
`create_and_run` creates the objects and adds them to the simulator.

Each `PholdSimulationObject` object is initialized with `args`, a namespace object that defines two attributes used by all objects:

* `args.num_phold_objects`: the number of PHOLD objects to run
* `args.frac_self_events`: the fraction of events sent to self

At time 0, each PHOLD object schedules an `InitMsg` event for itself that occurs after a random exponential time delay with mean = 1.0.

The `handle_simulation_event` method handles all events.
At each event, a PHOLD object schedules one more event.
It uses a U(0,1) random value to randomly schedule the event for itself (with probability `args.frac_self_events`) or for another PHOLD object.

If the event is being scheduled for another PHOLD object, this line obtains a reference to the object: 

    receiver = self.simulator.simulation_objects[str(obj_index)]

In DE-Sim, the attribute `self.simulator` always references the simulation engine that is running, and `self.simulator.simulation_objects` is a dictionary that maps simulation object names to simulation object instances.

The prediction generated by a simulation can be saved in many ways.
While illustrating more features of DE-Sim, this example just prints them.

Each event is recorded by `record_event`.
It accesses the DE-Sim `Event` object that's passed to all event handlers.
`de_sim.event.Event` provides five useful fields:

* `sending_object`: the object that created and sent the event
* `creation_time`: the simulation time when the event was created (a.k.a. its *send time*)
* `receiving_object`: the object that received the event
* `event_time`: the simulation time when the event must execute (a.k.a. its *receive time*)
* `message`: the `SimulationMessage` carried by the event

However, instead of the event's `event_time`, `record_event` uses `self.time` to report the simulation time when the event is being executed, as they are always equal.

In [33]:
from argparse import Namespace
args = Namespace(time_max=4,
                 frac_self_events=0.3,
                 num_phold_objects=6)
create_and_run(args)

Sender	Send	Receivr	Event	Message type
	time		time	
5	0.00	5	0.15	InitMsg
2	0.00	2	0.15	InitMsg
5	0.15	3	0.16	MessageSentToOtherObject
2	0.15	2	0.37	MessageSentToSelf
0	0.00	0	0.38	InitMsg
3	0.16	3	0.55	MessageSentToSelf
3	0.00	3	0.91	InitMsg
1	0.00	1	1.34	InitMsg
1	1.34	5	1.52	MessageSentToOtherObject
3	0.55	3	1.72	MessageSentToSelf
3	1.72	4	1.81	MessageSentToOtherObject
0	0.38	4	2.01	MessageSentToOtherObject
3	0.91	1	2.03	MessageSentToOtherObject
5	1.52	1	2.21	MessageSentToOtherObject
4	1.81	4	2.46	MessageSentToSelf
2	0.37	1	2.55	MessageSentToOtherObject
4	2.01	4	2.98	MessageSentToSelf
4	0.00	4	3.10	InitMsg
1	2.03	5	3.20	MessageSentToOtherObject
1	2.21	2	3.36	MessageSentToOtherObject
2	3.36	3	3.98	MessageSentToOtherObject
Executed 21 events.

